In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
#We found a data set with information on the expenditure on health per country
#as a percentage of the GDP

df_health=pd.read_csv("../data/expenditure_on_health_total_of_gdp.csv")

df_health.head()

,id,utc_created,utc_updated,country,country_code,indicator_code,indicator,value,year
0,1,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,53906,"Expenditure on health, total (% of GDP)",6.4,2005
1,2,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,53906,"Expenditure on health, total (% of GDP)",6.5,2006
2,3,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,53906,"Expenditure on health, total (% of GDP)",7.6,2007
3,4,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,53906,"Expenditure on health, total (% of GDP)",8.0,2009
4,5,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,53906,"Expenditure on health, total (% of GDP)",10.4,2010


In [11]:
df_h = df_health.drop(columns=["id",
                               "utc_created",
                               "utc_updated",
                               "country_code",
                               "indicator_code",
                               "indicator"])

df_h = df_h.rename(columns={"value":"%GDP_health"})
df_h.head()
df_h[df_h["country"] == "Albania"]

,country,%GDP_health,year
14,Albania,2.6,1995
15,Albania,6.4,2000
16,Albania,6.2,2005
17,Albania,6.0,2006
18,Albania,6.1,2007
19,Albania,6.3,2009
20,Albania,6.0,2010
21,Albania,6.3,2011


In [6]:
#We want to merge this new table with our master table
# Is there a correlation between expenditure on health and suicides rate?

In [8]:
df_suicides = pd.read_csv("../data/master_grouped_ccy.csv")
df_s = df_suicides.drop(columns="Unnamed: 0")
df_s.head()

,continent,country,year,suicides_no,population,gdp_per_capita ($),s_rate
0,Europe,Albania,1987,73,2709600,796,2.69
1,Europe,Albania,1988,63,2764300,769,2.28
2,Europe,Albania,1989,68,2803100,833,2.43
3,Europe,Albania,1992,47,2822500,251,1.67
4,Europe,Albania,1993,73,2807300,437,2.60


In [9]:
df_sh = df_s.merge(df_h, how="left", on=["country","year"])

In [13]:
df_sh.head()

,continent,country,year,suicides_no,population,gdp_per_capita ($),s_rate,%GDP_health
0,Europe,Albania,1987,73,2709600,796,2.69,NaN
1,Europe,Albania,1988,63,2764300,769,2.28,NaN
2,Europe,Albania,1989,68,2803100,833,2.43,NaN
3,Europe,Albania,1992,47,2822500,251,1.67,NaN
4,Europe,Albania,1993,73,2807300,437,2.60,NaN


In [ ]:
#Now we have our tables together, so we can do the linear regression

In [20]:
df_sh = df_sh[df_sh["%GDP_health"].notna()]
df_sh.head()

,continent,country,year,suicides_no,population,gdp_per_capita ($),s_rate,%GDP_health
6,Europe,Albania,1995,88,2903400,835,3.03,2.6
11,Europe,Albania,2000,54,2796300,1299,1.93,6.4
16,Europe,Albania,2005,0,2783320,2931,0.00,6.2
17,Europe,Albania,2006,0,2780176,3235,0.00,6.0
18,Europe,Albania,2007,124,2770344,3863,4.48,6.1


In [29]:
df_c = df_sh[df_sh["continent"] == "Europe"]
df_sh_cor = sm.add_constant(df_c)

df_sh_cor.head()

y = df_sh_cor["s_rate"]
X = df_sh_cor[["const","%GDP_health"]]
lin_reg = sm.OLS(y,X)
results = lin_reg.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 s_rate   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     3.424
Date:                Tue, 28 Apr 2020   Prob (F-statistic):             0.0652
Time:                        16:48:28   Log-Likelihood:                -1077.8
No. Observations:                 298   AIC:                             2160.
Df Residuals:                     296   BIC:                             2167.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          18.9300      2.340      8.090      0.000      14.325      23.535
%GDP_health    -0.5198      0.281     -1.850      0.065      -1.073       0.033
==============================================================================
Omnibus:                       26.787   Durbin-Watson:                   0.540
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.130
Skew:                           0.665   Prob(JB):                     3.88e-08
Kurtosis:                       3.989   Cond. No.                         37.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""